### 6.12 读取嵌套型和大小可变的二进制结构

In [4]:
polys = [
    [(1.0, 2.5), (3.5, 4.0), (2.5, 1.5)],
    [(7.0, 1.2), (5.1, 3.0), (0.5, 7.5), (0.8, 9.0)],
    [(3.4, 6.3), (1.2, 0.5), (4.6, 9.2)]
]

写入数据

In [7]:
import struct
import itertools

def write_polys(filename, polys):
    # Determine bounding box
    flattened = list(itertools.chain(*polys))
    min_x = min(x for x, y in flattened)
    max_x = max(x for x, y in flattened)
    min_y = min(y for x, y in flattened)
    max_y = max(y for x, y in flattened)
    
    with open(filename, 'wb') as f:
        f.write(struct.pack('<iddddi',
                           0x1234,
                            # 坐标最小值最大值
                           min_x,min_y,
                           max_x,max_y,
                            # 三角形数量
                           len(polys)))
        for poly in polys:
            size = len(poly) * struct.calcsize('<dd')
            f.write(struct.pack('<i', size+4))
            for pt in poly:
                f.write(struct.pack('<dd', *pt))
            
write_polys('polys.bin', polys)

读回数据

In [1]:
import struct

def read_polys(filename):
    with open(filename, 'rb') as f:
        # Read the header
        header = f.read(40)
        file_code, min_x, min_y, max_x, max_y, num_polys = \
            struct.unpack('<iddddi', header)
        polys = []
        for n in range(num_polys):
            pbytes, = struct.unpack('<i', f.read(4))
            poly = []
            for m in range(pbytes // 16):
                pt = struct.unpack('<dd', f.read(16))
                poly.append(pt)
            polys.append(poly)
    return polys

尽管上面代码能够工作，但其中混杂了一些read调用、对结构的解包以及其他一些细节，因此代码比较杂乱。  
我们下面逐步构建出一个用来解释二进制数据的高级解决方案，让程序员提供文件格式的高层规范，而将读取文件以及解包所有数据的细节部分隐藏起来。

当读取二进制数据时，文件中包含文件头和其他数据结构是非常常见的。  
sturct模块能将数据解包为元组，但还可以通过类。如下所示：

In [12]:
import struct

class StructField:
    """
    Descriptor representing a simple structure field
    """
    def __init__(self, formats, offset):
        self.formats = formats
        self.offset = offset
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            r = struct.unpack_from(self.formats, instance._buffer, self.offset)
            return r[0] if len(r) == 1 else r
        
class Structure:
    def __init__(self, bytedata):
        self._buffer = memoryview(bytedata)

In [13]:
class PolyHeader(Structure):
    file_code = StructField('<i', 0)
    min_x = StructField('<d', 4)
    min_y = StructField('<d', 12)
    max_x = StructField('<d', 20)
    max_y = StructField('<d', 28)
    num_polys = StructField('<i', 36)
                                                                                                                                                                                                                                                                                                                                                                                                                        

In [14]:
f = open('polys.bin', 'rb')
phead = PolyHeader(f.read(40))
phead.file_code == 0x1234

True

In [15]:
phead.min_x

0.5

尽管上面得到了便利的类接口，但代码比较冗长，需要用户指定许多底层细节。  
得到的结果中，这个类也缺少一些常用的便捷方法。  
任何时候当面对这种过于冗长的类定义时，都应该考虑使用类装饰器或着元类。

In [16]:
class StructureMeta(type):
    """
    Metaclass that automatically creates StructField descriptors
    """
    def __init__(self, clsname, bases, clsdict):
        fields = getattr(self, '_fields_', [])
        byte_order = ''
        offset = 0
        for formats, fieldname in fields:
            if formats.startswith(('<','>','!','@')):
                byte_order = formats[0]
                formats = formats[1:]
            formats = byte_order + formats
            setattr(self, fieldname, StructField(formats, offset))
            offset += struct.calcsize(formats)
        setattr(self, 'struct_size', offset)
        
class Structure(metaclass=StructureMeta):
    def __init__(self, bytedata):
        self._buffer = bytedata
        
    @classmethod
    def from_file(cls, f):
        return cls(f.read(cls.struct_size))
            

In [17]:
class PolyHeader(Structure):
    _fields_=[
        ('<i', 'file_code'),
        ('d', 'min_x'),
        ('d', 'min_y'),
        ('d', 'max_x'),
        ('d', 'max_y'),
        ('i', 'num_polys')
    ]

In [18]:
f = open('polys.bin', 'rb')
phead = PolyHeader.from_file(f)
phead.file_code == 0x1234

True

一旦引入了元类，就可以为其构建更多智能化的操作。  
假如想对嵌套型的二级制结构提供支持。

In [ ]:
class NestedStruct:
    """
    Descriptor representing a nested structure
    """
    def __init__(self, name, struct_type, offset):
        